# Scraps

In [ ]:
subset = full_cleaned[full_cleaned['price'] < 950000]
print('Percent removed:',(len(full_cleaned) - len(subset))/len(full_cleaned)) 

In [ ]:
#look at sqft_living values
raw_df.groupby(['zipcode']).agg(['median','mean', 'count'])

In [ ]:
#homoskadaticy check 
plt.scatter(model.predict(train[x_cols]), model.resid)
plt.plot(model.predict(train[x_cols]), [0 for i in range(len(train))])

In [ ]:
# Extract the p-value table from the summary and use it to subset our features
summary = model.summary()
p_table = summary.tables[1]
p_table = pd.DataFrame(p_table.data)
p_table.columns = p_table.iloc[0]
p_table = p_table.drop(0)
p_table = p_table.set_index(p_table.columns[0])
p_table['P>|t|'] = p_table['P>|t|'].astype(float)
x_cols = list(p_table[p_table['P>|t|'] < 0.05].index)
x_cols.remove('Intercept')
print(len(p_table), len(x_cols))
print(x_cols[:5])
p_table.head()

In [ ]:
#checking for linearity of features 
sns.jointplot('price','bathrooms', data=cleaned, kind='reg'); 
sns.jointplot('price','bedrooms', data=cleaned, kind='reg'); 
sns.jointplot('price','condition', data=cleaned, kind='reg');  
sns.jointplot('price','grade', data=cleaned, kind='reg'); 
sns.jointplot('price','sqft_living', data=cleaned, kind='reg'); 
sns.jointplot('price','floors', data=cleaned, kind='reg'); 
sns.jointplot('price','sqft_above', data=cleaned, kind='reg');  
sns.jointplot('price','sqft_living15', data=cleaned, kind='reg'); 
#checking for multicollinearity  
feats = ['bathrooms', 'bedrooms', 'condition', 'grade', 'sqft_living', 'floors',  'sqft_above', 'sqft_living15']
corr = df[feats].corr()
corr 

#considering r > 0.65 as multicolinear bathrooms & grade appears to be the only multicolinear item?   
sns.heatmap(corr, center=0, annot=True) 
Initial Model
I would like to look at what renovatable (changable) feature most affects housing price. I am choosing to look only at 
features that could be altered i.e. not view, zipcode, waterfront, yr_built, zipcode, lat, long, sqft_lot15 since those
features cannot be changed. I am modeling my data this way because my stake holder in this project is a nonprofit/ city
partnership looking to remodel houses at affordable prices & build equity in communities. 

#The problem
outcome = 'price'
x_cols = ['bathrooms', 'bedrooms', 'condition', 'grade', 'sqft_living', 'floors',  'sqft_above', 'sqft_living15']  

#create a seprate df before changing values 
inital_df = df.copy()

for col in x_cols: 
    inital_df[col] = (inital_df[col] - df[col].mean())/inital_df[col].std()
inital_df.head() 

#fitting the model 
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors 
model = ols(formula=formula, data=inital_df).fit()
model.summary() 

#variance inflation factor  
X = inital_df[x_cols] 
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])] 
list(zip(x_cols, vif)) 

#update model 
outcome = 'price'
x_cols =['bathrooms', 'bedrooms', 'condition', 'grade', 'sqft_living', 'floors',  'sqft_above', 'sqft_living15'] 
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors 
model = ols(formula=formula, data=inital_df).fit()
model.summary() 

#pval of 0.0 means that we are certain this relationship is not due to chance.  
Normalicy Check 
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True) 
Homoscedacity Check 
plt.scatter(model.predict(inital_df[x_cols]), model.resid)
plt.plot(model.predict(inital_df[x_cols]), [0 for i in range(len(inital_df))]) 
Model Refinement
#data cut off point
for i in range(90, 99): 
    q = i / 100 
    print('{} percentile: {}'.format(q, inital_df['price'].quantile(q=q))) 
subset = inital_df[inital_df['price'] < 1260000]
print('Percent removed:', (len(inital_df) - len(subset)) / len(df))
outcome= 'price'
x_cols = ['bathrooms', 'bedrooms', 'condition', 'grade', 'sqft_living', 'floors',  'sqft_above', 'sqft_living15'] 
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=subset).fit()
model.summary() 

#recheck normality 
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True) 

#recheck homostatisity 
plt.scatter(model.predict(subset[x_cols]), model.resid)
plt.plot(model.predict(subset[x_cols]), [0 for i in range(len(subset))])

In [ ]:
#Check for correlation numerically
subset7=X_train.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
subset['pairs'] = list(zip(mldf.level_0, mldf.level_1))

# set index to pairs
corr.set_index(['pairs'], inplace = True)

#d rop level columns
corr.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
corr.columns = ['cc']

# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
# for the sake of exercise, kept it in.
corr.drop_duplicates(inplace=True)

In [ ]:
mldf[(mldf.cc>.75) & (mldf.cc <1)]

In [ ]:
#baseline model 

regression = LinearRegression()
crossvalidation = KFold(n_splits=3, shuffle=True, random_state=1) 
    
baseline= np.mean(cross_val_score(regression, mldf, mldf['price'], scoring='r2', cv=crossvalidation))
print("Baseline:", baseline)

In [ ]:
X = z_score.drop('price', axis=1)
y = z_score['price'] 
# Split the data into training and test sets (assign 20% to test set) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10) 

In [ ]:
# Split the data into training and test sets (assign 20% to test set) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [ ]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
X_train

In [ ]:
X_train.columns

In [ ]:
X1 = X_train
y1 = y_train

In [ ]:
train = pd.concat([X_train, y_train], axis=1, join='outer')

Begin removing features that are overly correlated as indicated on the df.cc check above.

### Trial 2 
Remove 'sqft_living15', 'sqft_above', 'sqft_living', 'lot'

In [ ]:
train1 = train.drop(["sqft_living15", "sqft_above", "sqft_living", "lot"], axis=1)

In [ ]:
#redefine problem
outcome = "price"
t2 = ['bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'view',
       'condition', 'grade', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_lot15',
       'wf', 'living', 'above', 'fliving', 'flot', 'basement']

In [ ]:
lin_reg(outcome="price", x_cols=t2, df=train1)

In [ ]:
correlation_df2 = train1.corr() 
correlation_df2

In [ ]:
sns.heatmap(correlation_df2, center=0, annot=True) 

In [ ]:
df[(df.cc>.75) & (df.cc <1)]

### Trial 3 

In [ ]:
train2 = train.drop(["living"], axis=1)

In [ ]:
train2.columns

In [ ]:
#redefine problem
outcome = "price"
t3 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view',
       'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       'wf', 'lot', 'above', 'fliving', 'flot', 'basement']

In [ ]:
lin_reg(outcome=outcome, x_cols=t3, df=train2)

That improved our model!

In [ ]:
correlation_df3 = train2.corr() 
correlation_df3

In [ ]:
sns.heatmap(correlation_df2, center=0, annot=True) 

In [ ]:
#Check for correlation numerically
df1=train2.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df1['pairs'] = list(zip(df1.level_0, df1.level_1))

# set index to pairs
df1.set_index(['pairs'], inplace = True)

#d rop level columns
df1.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
df1.columns = ['cc']

# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
# for the sake of exercise, kept it in.
df1.drop_duplicates(inplace=True)

In [ ]:
df1[(df1.cc>.75) & (df1.cc <1)]

### Trail 4  

In [ ]:
train3 = train2.drop(['sqft_living15', 'sqft_above', 'lot'], axis=1)

In [ ]:
train3.columns

In [ ]:
#redefine problem
outcome = "price"
t4 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view',
       'condition', 'grade', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'sqft_lot15', 'wf', 'above', 'fliving',
       'flot', 'basement']

In [ ]:
lin_reg(outcome="price", x_cols=t4, df=train3)

In [ ]:
#Check for correlation numerically
df2=train3.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df2['pairs'] = list(zip(df2.level_0, df2.level_1))

# set index to pairs
df2.set_index(['pairs'], inplace = True)

#d rop level columns
df2.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
df2.columns = ['cc']

# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
# for the sake of exercise, kept it in.
df2.drop_duplicates(inplace=True) 

df2[(df2.cc>.75) & (df2.cc <1)]

### Trial 5 

In [ ]:
train4 = train3.drop(["above", "sqft_lot15"], axis=1)

In [ ]:
train4.columns

In [ ]:
outcome5 = "price"
t5 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view',
       'condition', 'grade', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'wf', 'fliving', 'flot', 'basement']

In [ ]:
lin_reg(outcome="price", x_cols=t5, df=train4)

In [ ]:
#Check for correlation numerically
df3=train4.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df3['pairs'] = list(zip(df3.level_0, df3.level_1))

# set index to pairs
df3.set_index(['pairs'], inplace = True)

#d rop level columns
df3.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
df3.columns = ['cc']

# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
# for the sake of exercise, kept it in.
df3.drop_duplicates(inplace=True) 

df3[(df3.cc>.75) & (df3.cc <1)]

All previous trails were dropping columns based on high correlation scores. 

### Trial 6 

In [ ]:
train5 = train4.copy()

In [ ]:
train5.columns

In [ ]:
outcome= "price"
t6 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view',
       'condition', 'grade', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'wf', 'fliving', 'flot', 'basement']

In [ ]:
lin_reg(outcome="price", x_cols=t6, df=train5)

### Trail 7 
Look for correlated columns to find a model with signifigance.

In [ ]:
train6 = train5.drop(["zipcode"], axis=1)

In [ ]:
train6.columns

In [ ]:
#look for correlated columns
columns_correlations = []
columns_non_numeric = []

for column in mldf.drop(columns=[outcome]).columns:
    try:
        corr = np.abs(mldf[column].corr(mldf[outcome]))
        t = (column, corr)
        columns_correlations.append(t)
    except:
        columns_non_numeric.append(column)  
        
correlated_features_above_2 = [t[0] for t in columns_correlations if t[1] >= 0.10]
correlated_features_above_2

In [ ]:
columns_correlations = []
columns_non_numeric = []

for column in train6.drop(columns=[outcome]).columns:
    try:
        corr = np.abs(train6[column].corr(train6[outcome]))
        t = (column, corr)
        columns_correlations.append(t)
    except:
        columns_non_numeric.append(column) 
columns_correlations

In [ ]:
correlated_features_above_2 = [t[0] for t in columns_correlations if t[1] >= 0.10]
correlated_features_above_2

In [ ]:
outcome="price" 
t8 = ['bedrooms', 'bathrooms','floors', 'grade', 'lat','long','fliving', 'basement']

In [ ]:
lin_reg(outcome="price", x_cols=t8, df=train4)

This is my more correlated but more predictive model I think give there is a time and a place for this type of model it will be more predictived even though it is multicolinear. You could use both models in a buisness setting. 

In [ ]:
#take the sum of bathrooms/bedrooms 
train4["bed_bath"] = train4['bedrooms'] + train4['bathrooms'] 

In [ ]:
outcome="price"
t7 = ['fliving', 'grade', 'flot']

I iterated over the combination of statistically signifigance correlations dozens of times. Most combinations hovered in the high 0.3 for the R2 scored. Many of the features in this data set were heavily correlated. I landed on the model below in part consideration of my stakehold looking at correlations that can be modified(renovated) in order to predict future home price. While it is not a perfect model I feel confident that any predictions I do make will not be do to multicollinearity and meets the assumptions of OLS. 

In [ ]:
outcome="price"
t7 = ['fliving', 'grade', 'flot']

In [ ]:
lin_reg(outcome="price", x_cols=t7, df=train6)

### Fit Model

In [ ]:
linreg = LinearRegression()

linreg.fit(X_train, y_train)
y_hat_test = linreg.predict(X_test)

In [ ]:
test = pd.concat([X_test, y_test], axis=1, join='outer')

In [ ]:
train = pd.concat([X_train, y_train], axis=1, join='outer')

### Mean Squared Error

In [ ]:
y_hat_test

In [ ]:
test_residuals = y_hat_train - y_test

test_mse = mean_squared_error(y_test, y_hat_train)
test_mse 
#sq root of this is 105837.68

In [ ]:
test_residuals = y_hat_train - y_test

test_mse = mean_squared_error(y_test, y_hat_train)
test_mse 
#sq root of this is 105837.68

### KFold

In [ ]:
mse = make_scorer(mean_squared_error)

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse) 
cv_5_results

In [ ]:
cv_5_results  = np.mean(cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error'))
cv_10_results = np.mean(cross_val_score(linreg, X, y, cv=10, scoring='neg_mean_squared_error'))
cv_20_results = np.mean(cross_val_score(linreg, X, y, cv=20, scoring='neg_mean_squared_error')) 

print(cv_5_results)
print(cv_10_results)
print(cv_20_results)

In [ ]:
All of these are relatively similar which is good! 

In [ ]:
fig = px.imshow(tts2)
fig.show()

In [ ]:
sns.heatmap(tts2, center=0, annot=True) 

In [ ]:
subset6.dtypes

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df = tts2

# Split data into training and test splits
train_idx, test_idx = train_test_split(df.index, test_size=.25, random_state=0)
df['split'] = 'train'
df.loc[test_idx, 'split'] = 'test'

X = df[['view', 'grade']]
y = df['price']
X_train = df.loc[train_idx, ['view', 'grade']]
y_train = df.loc[train_idx, 'price']

# Condition the model on sepal width and length, predict the petal width
model = LinearRegression()
model.fit(X_train, y_train)
df['prediction'] = model.predict(X)

fig = px.scatter(
    df, x='price', y='prediction',
    marginal_x='histogram', marginal_y='histogram',
    color='split', trendline='ols'
)
fig.update_traces(histnorm='probability', selector={'type':'histogram'})
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)

fig.update_layout( 
    uniformtext_minsize=8, uniformtext_mode='hide',
    title={
        'text': "Sqft_living & Grade", 
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
    
    xaxis=dict(
        title='price',
        titlefont_size=18,
        tickfont_size=14, 
    ), 
    yaxis=dict(
        title='prediction',
        titlefont_size=18,
        tickfont_size=14,
    ),
    
    #paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
   
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate. 
) 

fig.show()

In [ ]:
df = mldf
X = mldf.price[:, None]
X_train, X_test, y_train, y_test = train_test_split(X, mldf, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

x_range = np.linspace(X.min(), X.max(), 100)
y_range = model.predict(x_range.reshape(-1, 1))


fig = go.Figure([
    go.Scatter(x=X_train.squeeze(), y=y_train, name='train', mode='markers'),
    go.Scatter(x=X_test.squeeze(), y=y_test, name='test', mode='markers'),
    go.Scatter(x=x_range, y=y_range, name='prediction')
])
fig.show()

In [ ]:
fm_viz = mldf.copy()

In [ ]:
fm_viz_no200k = fm_viz.loc[fm_viz['sqft_lot15'] < 50000]

In [ ]:
fm_viz_no200k['sqft_lot15'].unique

In [ ]:
df = fm_viz_no200k
fig = px.scatter(df, x="sqft_above", y="price",  facet_col="grade", color="view", trendline="ols")
fig.show()

results = px.get_trendline_results(fig)
print(results)

In [ ]:
subset6